# Final Project
## Red Wine Quality


In [53]:
import utils

table = utils.read_table('red_wine_quality.csv')
header = table[0]
table = table[1:]



## K Means Clustering 

1. Choose a value of k.
2. Select k objects in an arbitrary fashion. Use these as the initial set of k centroids.
3. Assign each of the objects to the cluster for which it is nearest to the centroid.
4. Recalculate the centroids of the k clusters.
5. Repeat steps 3 and 4 until the centroids no longer move.

In [58]:
import random
import math
import copy
import numpy
import matplotlib.pyplot as plt

#TODO: do I include the classification or not??? 
def compute_distance(v1, v2):
    """computes the distance between v1 and v2 using Eucildean distance. Does not include the classification attribute."""
    #assert(len(v1) == len(v2))
    #print("V1: ", v1)
    #print("V2: ", v2)
    dist = math.sqrt(sum([(v1[i] - v2[i]) ** 2 for i in range(len(v2)-1)]))
    return dist

def select_init_centroids(k, table):
    """select k instances at random to use as the initial centroids.""" 
    init_centroids = []
    for i in range(k):
        instance_index = random.randrange(len(table))
        while table[instance_index] in init_centroids:
            instance_index = random.randrange(len(table))
        init_centroids.append(table[instance_index])
    #print("init_centroids", init_centroids)
    return init_centroids

def append_distance(table, distances, point):
    """appends the distance between row in table and point to distances."""
    for i, row in enumerate(table):
        distance = compute_distance(row, point)
        distances[i].append(distance)
        
def find_centroid(distances):
    """gets the centroid index of smallest distance and appends it to distances."""
    for row in distances:
        cluster_index = row.index(min(row))
        row.append(cluster_index)

def combine_tables(table1, table2):
    """combines table 1 and table 2 together."""
    new_table = []
    assert(len(table1) == len(table2))
    for i, row in enumerate(table1):
        new_row = row + table2[i]
        new_table.append(new_row)
    return new_table

def compute_average(cluster, atts_range):
    """atts_range: is the range [0-att_range) in which to compute averages."""
    centroid = []
    for i in range(atts_range):
        column = utils.get_column(cluster, i)
        att_average = numpy.mean(column)
        centroid.append(round(att_average,2))
    return centroid
        
def recalculate_centroids(table, distances_table, att_list):
    # Combine and group by cluster 
    new_table = combine_tables(table, distances_table)
    cluster_index = len(new_table[0])-1
    group_names, groups = utils.group_by(new_table, cluster_index)
    
    # For each cluster
    new_centroids = []
    for cluster in groups:
        # get the average of each attribute
        new_centroid = compute_average(cluster, len(table[0]))
        new_centroids.append(new_centroid)
    return new_centroids
    
def compare_centroids(centroids1, centroids2):
    assert(len(centroids1) == len(centroids2))
    for i in range(len(centroids1)):
        if centroids1[i] != centroids2[i]:
            return False
    return True 

def k_means_clustering(k, table):
    # Select k objects in an arbitrary fashion. Use these as the initial set of k centroids.
    centroids = select_init_centroids(k, table)
    #print('Initial Centroids: ', centroids)
    
    match = False
    while match == False: 
        
        # Compute the distances of each instance to each centroid
        distances_table = [ [] for i in range(len(table)) ]
        for centroid in centroids:
            append_distance(table, distances_table, centroid)

        # Find the biggest distance and assign instance to that centroid
        find_centroid(distances_table)
        
        #print("Distances Table:")
        #utils.pretty_print(distances_table)

        # Recalculate the centroids by getting the mean of each cluster
        new_centroids = recalculate_centroids(table, distances_table, [])
        #print("New Centroids: ", new_centroids)

        # Check to see if the centroids have converged
        match = compare_centroids(centroids, new_centroids)
        centroids = new_centroids
        print("New Centroids: ", centroids)

    # Now we know what instance belongs to what cluster
    # these are "parallel tables" 
    #TODO: maybe return here or something, got to choose the "best" cluster
    #TODO: don't call this here
    
    score = objective_function(table, distances_table, centroids)
    return score, distances_table, centroids
    # Return the clusters as a list of tables
    #for i, row in enumerate(table): 
    #    row.append(distances_table)
    
# TODO: change this so accepts clusters and centroids, and doesn't have to combine anyhting
def objective_function(table, cluster_table, centroids):
    # cluster quality - through the use of an objective function, choose the k with the "best" value of objective func
    # total sum of squares - how clustered are the points to the centroid? compute for each cluster, sum over clusters
    
    # Combine and group by cluster 
    new_table = combine_tables(table, cluster_table)
    cluster_index = len(new_table[0])-1
    group_names, groups = utils.group_by(new_table, cluster_index)
    #print("Group Names: ", group_names)
    
    # for each cluster, compute the sum of squares
    # add these to a cluster_total of all clusters
    total_cluster_score = 0
    for i, cluster in enumerate(groups):
        distances = []
        for row in cluster:
            #print("Row: ", row)
            #print("Centroid:", centroids[row[cluster_index]])
            distance = compute_distance(row, centroids[row[cluster_index]])
            distances.append(distance)
        total_cluster_score += sum(distances)
    print(total_cluster_score)
    return total_cluster_score
    
def majority_voting(cluster, classification_index):
    # get the frequency of the classfication index
    values, counts = utils.get_frequencies(cluster, classification_index)
    print("values: ", values)
    print("Counts: ")
    
    # get the biggest one 
    highest_freq_index = counts.index(max(counts))
    
    # return that classification
    return values[highest_freq_index]
    
def predict(random_instance, centroids, clusters, classification_index):
    distances = []
    for centroid in centroids:
        distance = compute_distance(random_instance, centroid)
        distances.append(distance)
    cluster_index = distances.index(min(distances))
    
    majority_classification = majority_voting(clusters[cluster_index], classification_index)
    print("Random Instance: ", random_instance)
    print("Majority Classification: ", majority_classification)
    
def find_best_cluster(table, minimum, maximum):
    objective_scores = []
    x_axis = []
    for i in range(minimum, maximum+1):
        print(i)
        score, cluster_table, centroids = k_means_clustering(i, table)
        objective_scores.append(score)
        x_axis.append(i)
    #print(objective_scores)
    plt.figure()
    plt.plot(x_axis, objective_scores)
    plt.show()
        
#find_best_cluster(table, 3, 10)
#From plotting these objective functions scores, it appears that k=7 is an appropriate value.
score, distances_table, centroids = k_means_clustering(3, table)
# create a cluster table
cluster_table = copy.deepcopy(table)
for i, row in enumerate(cluster_table):
    cluster_table[i].append(distances_table[i][-1])
#utils.pretty_print(cluster_table)
print(centroids)
# group by cluster
group_names, groups = utils.group_by(cluster_table, len(cluster_table[0])-1)
#print(group_names)
#utils.pretty_print(cluster_table)

# predicting
random_instance = [8.9,0.22,0.48,1.8,0.077,29,60,0.9968,3.39,0.53,9.4,6]
predict(random_instance, centroids, groups, header.index("quality"))
#utils.pretty_print(table)



New Centroids:  [[8.03, 0.55, 0.28, 2.9, 0.09, 25.01, 89.11, 1.0, 3.3, 0.66, 10.11, 5.4], [8.61, 0.52, 0.29, 2.42, 0.09, 5.22, 13.2, 1.0, 3.3, 0.63, 10.72, 5.71], [8.38, 0.52, 0.26, 2.38, 0.08, 14.52, 34.74, 1.0, 3.32, 0.67, 10.5, 5.74]]
New Centroids:  [[8.03, 0.55, 0.28, 2.96, 0.09, 25.78, 92.56, 1.0, 3.3, 0.66, 10.08, 5.38], [8.73, 0.52, 0.3, 2.48, 0.09, 6.47, 16.22, 1.0, 3.29, 0.64, 10.66, 5.73], [8.22, 0.52, 0.25, 2.34, 0.09, 16.29, 39.8, 1.0, 3.33, 0.66, 10.46, 5.72]]
New Centroids:  [[8.07, 0.55, 0.29, 3.06, 0.09, 26.26, 96.78, 1.0, 3.29, 0.66, 10.08, 5.38], [8.66, 0.51, 0.29, 2.43, 0.08, 7.28, 18.15, 1.0, 3.3, 0.65, 10.66, 5.77], [8.16, 0.53, 0.24, 2.36, 0.09, 17.76, 44.03, 1.0, 3.33, 0.66, 10.41, 5.66]]
New Centroids:  [[8.03, 0.56, 0.29, 3.12, 0.09, 26.14, 100.29, 1.0, 3.29, 0.66, 10.07, 5.36], [8.59, 0.52, 0.28, 2.4, 0.08, 7.91, 19.55, 1.0, 3.3, 0.65, 10.62, 5.76], [8.18, 0.53, 0.25, 2.39, 0.09, 19.07, 47.43, 1.0, 3.33, 0.67, 10.39, 5.65]]
New Centroids:  [[7.97, 0.57, 0.28,

## From plotting these objective functions scores, it appears that k=7 is an appropriate value.
